In [ ]:
import numpy as np
import pylab as pl
import scipy
import matplotlib.pyplot as plt

import quicklens as ql

import mlcmb

configpath=mlcmb.config.mlcmbpath+"/mlcmb/configs/config_128_qu.ini" #"/mlcmb/configs/config_epsilonboost.ini" #"/mlcmb/configs/config_master.ini
params = mlcmb.config.Parameters(configpath)

datasetid = params.datasetid

#for combi plot:
#mlcmb.config.folder_path_run = mlcmb.config.datapath+"runs/n128_qu_j4_adam04_500epochs/"

In [ ]:
#load test input data 
data_test_input = np.load(params.datapath+"datasets/dataset_wf_test_"+str(datasetid)+".npy")
#load test output data
fname_results = params.folder_path_run+"dataset_wf_test_"+str(datasetid)+"_results.npy"
data_test_result = np.load(fname_results)

nsims = data_test_result.shape[0]
print (data_test_result.shape)

#also make EB from it
data_test_result_eb = np.zeros_like(data_test_result)

In [ ]:
print (fname_results)

## Calculate metrics

In [ ]:
#calculate mapWF correlation coeff
nx = params.nx
dx = params.dx
lmax       = 4600 #params.lmax #5000 3000 #3500 #7000
lbins      = np.linspace(100, lmax, 100) #np.linspace(100, lmax, 30)       # multipole bins.
ell_binned = lbins[:-1] + np.diff(lbins)

#mapWF
corr_coeff_nn_avg_q = np.zeros(ell_binned.shape[0])
corr_coeff_obs_avg_q = np.zeros(ell_binned.shape[0])
auto_nn_avg_q = np.zeros(ell_binned.shape[0])
auto_obs_avg_q = np.zeros(ell_binned.shape[0])
auto_true_avg_q = np.zeros(ell_binned.shape[0])
diffpower_nn_avg_q = np.zeros(ell_binned.shape[0])
diffpower_obs_avg_q = np.zeros(ell_binned.shape[0])

corr_coeff_nn_avg_u = np.zeros(ell_binned.shape[0])
corr_coeff_obs_avg_u = np.zeros(ell_binned.shape[0])
auto_nn_avg_u = np.zeros(ell_binned.shape[0])
auto_obs_avg_u = np.zeros(ell_binned.shape[0])
auto_true_avg_u = np.zeros(ell_binned.shape[0])
diffpower_nn_avg_u = np.zeros(ell_binned.shape[0])
diffpower_obs_avg_u = np.zeros(ell_binned.shape[0])

corr_coeff_nn_avg_e = np.zeros(ell_binned.shape[0])
corr_coeff_obs_avg_e = np.zeros(ell_binned.shape[0])
auto_nn_avg_e = np.zeros(ell_binned.shape[0])
auto_obs_avg_e = np.zeros(ell_binned.shape[0])
auto_true_avg_e = np.zeros(ell_binned.shape[0])
diffpower_nn_avg_e = np.zeros(ell_binned.shape[0])
diffpower_obs_avg_e = np.zeros(ell_binned.shape[0])

corr_coeff_nn_avg_b = np.zeros(ell_binned.shape[0])
corr_coeff_obs_avg_b = np.zeros(ell_binned.shape[0])
auto_nn_avg_b = np.zeros(ell_binned.shape[0])
auto_obs_avg_b = np.zeros(ell_binned.shape[0])
auto_true_avg_b = np.zeros(ell_binned.shape[0])
diffpower_nn_avg_b = np.zeros(ell_binned.shape[0])
diffpower_obs_avg_b = np.zeros(ell_binned.shape[0])


for map_id in range(nsims): 
    ######### Q maps
    mapWF_true_map_q = data_test_input[map_id,:,:,14] 
    mapNN_map_q = data_test_result[map_id,:,:,0]
    mapObs_map_q = data_test_input[map_id,:,:,10]  

    #make these rmaps and get cffts from which we can get cls and mls
    mapWF_true_cfft_q = ql.maps.rmap(nx, dx,map=mapWF_true_map_q).get_cfft()
    mapNN_cfft_q = ql.maps.rmap(nx, dx,map=mapNN_map_q).get_cfft()
    mapObs_cfft_q = ql.maps.rmap(nx, dx,map=mapObs_map_q).get_cfft()
    
    #cross powers
    cross_map_cfft_nn_q = ql.maps.cfft( nx, dx, fft=(mapNN_cfft_q.fft * np.conj(mapWF_true_cfft_q.fft)) )
    cross_power_nn_q = cross_map_cfft_nn_q.get_ml(lbins)  #use ml because the cfft already is a power/multiple of two maps    
    cross_map_cfft_obs_q = ql.maps.cfft( nx, dx, fft=(mapObs_cfft_q.fft * np.conj(mapWF_true_cfft_q.fft)) )
    cross_power_obs_q = cross_map_cfft_obs_q.get_ml(lbins)  #use ml because the cfft already is a power/multiple of two maps    
    
    #auto powers
    auto_true_q = mapWF_true_cfft_q.get_cl(lbins) #use cl because we really want the power of this map
    auto_nn_q = mapNN_cfft_q.get_cl(lbins)
    auto_obs_q = mapObs_cfft_q.get_cl(lbins)
    auto_true_avg_q += auto_true_q.cl.real
    auto_nn_avg_q += auto_nn_q.cl.real
    auto_obs_avg_q += auto_obs_q.cl.real
    
    #corr coeff from spectra
    corr_coeff_nn_q = cross_power_nn_q.specs['cl']/(auto_nn_q.specs['cl']*auto_true_q.specs['cl'])**(1./2) 
    corr_coeff_nn_avg_q += corr_coeff_nn_q.real
    corr_coeff_obs_q = cross_power_obs_q.specs['cl']/(auto_obs_q.specs['cl']*auto_true_q.specs['cl'])**(1./2) 
    corr_coeff_obs_avg_q += corr_coeff_obs_q.real
        
    #difference maps
    diff_nn_cfft_q = mapNN_cfft_q - mapWF_true_cfft_q  
    diff_obs_cfft_q = mapObs_cfft_q - mapWF_true_cfft_q 
    diffpower_nn_q = diff_nn_cfft_q.get_cl(lbins) #use cl because we really want the power of this map
    diffpower_obs_q = diff_obs_cfft_q.get_cl(lbins)
    diffpower_nn_avg_q += diffpower_nn_q.cl.real
    diffpower_obs_avg_q += diffpower_obs_q.cl.real    
    
    
    ######### U maps
    mapWF_true_map_u = data_test_input[map_id,:,:,15] 
    mapNN_map_u = data_test_result[map_id,:,:,1]
    mapObs_map_u = data_test_input[map_id,:,:,11]  

    #make these rmaps and get cffts from which we can get cls and mls
    mapWF_true_cfft_u = ql.maps.rmap(nx, dx,map=mapWF_true_map_u).get_cfft()
    mapNN_cfft_u = ql.maps.rmap(nx, dx,map=mapNN_map_u).get_cfft()
    mapObs_cfft_u = ql.maps.rmap(nx, dx,map=mapObs_map_u).get_cfft()
    
    #cross powers
    cross_map_cfft_nn_u = ql.maps.cfft( nx, dx, fft=(mapNN_cfft_u.fft * np.conj(mapWF_true_cfft_u.fft)) )
    cross_power_nn_u = cross_map_cfft_nn_u.get_ml(lbins)  #use ml because the cfft already is a power/multiple of two maps    
    cross_map_cfft_obs_u = ql.maps.cfft( nx, dx, fft=(mapObs_cfft_u.fft * np.conj(mapWF_true_cfft_u.fft)) )
    cross_power_obs_u = cross_map_cfft_obs_u.get_ml(lbins)  #use ml because the cfft already is a power/multiple of two maps    
    
    #auto powers
    auto_true_u = mapWF_true_cfft_u.get_cl(lbins) #use cl because we really want the power of this map
    auto_nn_u = mapNN_cfft_u.get_cl(lbins)
    auto_obs_u = mapObs_cfft_u.get_cl(lbins)
    auto_true_avg_u += auto_true_u.cl.real
    auto_nn_avg_u += auto_nn_u.cl.real
    auto_obs_avg_u += auto_obs_u.cl.real
    
    #corr coeff from spectra
    corr_coeff_nn_u = cross_power_nn_u.specs['cl']/(auto_nn_u.specs['cl']*auto_true_u.specs['cl'])**(1./2) 
    corr_coeff_nn_avg_u += corr_coeff_nn_u.real
    corr_coeff_obs_u = cross_power_obs_u.specs['cl']/(auto_obs_u.specs['cl']*auto_true_u.specs['cl'])**(1./2) 
    corr_coeff_obs_avg_u += corr_coeff_obs_u.real
        
    #difference maps
    diff_nn_cfft_u = mapNN_cfft_u - mapWF_true_cfft_u  
    diff_obs_cfft_u = mapObs_cfft_u - mapWF_true_cfft_u 
    diffpower_nn_u = diff_nn_cfft_u.get_cl(lbins) #use cl because we really want the power of this map
    diffpower_obs_u = diff_obs_cfft_u.get_cl(lbins)
    diffpower_nn_avg_u += diffpower_nn_u.cl.real
    diffpower_obs_avg_u += diffpower_obs_u.cl.real  
    
    
    
    ######### E maps
    
    mapWF_true_map_e = data_test_input[map_id,:,:,12]  
    mapObs_map_e = data_test_input[map_id,:,:,8] 
    
    #convert NN output QU to EB
    tqumap = ql.maps.tqumap( nx, dx, maps=[np.zeros((nx,nx)),mapNN_map_q,mapNN_map_u] )
    tebfft = tqumap.get_teb()
    data_test_result_eb[map_id,:,:,0] = tebfft.get_ffts()[1].get_rmap().map
    data_test_result_eb[map_id,:,:,1] = tebfft.get_ffts()[2].get_rmap().map
    
    #make these rmaps and get cffts from which we can get cls and mls
    mapWF_true_cfft_e = ql.maps.rmap(nx, dx,map=mapWF_true_map_e).get_cfft()
    #mapNN_cfft_e = ql.maps.rmap(nx, dx,map=mapNN_map_e).get_cfft()
    mapNN_cfft_e = tebfft.get_cffts()[1]
    mapObs_cfft_e = ql.maps.rmap(nx, dx,map=mapObs_map_e).get_cfft()
    
    #cross powers
    cross_map_cfft_nn_e = ql.maps.cfft( nx, dx, fft=(mapNN_cfft_e.fft * np.conj(mapWF_true_cfft_e.fft)) )
    cross_power_nn_e = cross_map_cfft_nn_e.get_ml(lbins)  #use ml because the cfft already is a power/multiple of two maps    
    cross_map_cfft_obs_e = ql.maps.cfft( nx, dx, fft=(mapObs_cfft_e.fft * np.conj(mapWF_true_cfft_e.fft)) )
    cross_power_obs_e = cross_map_cfft_obs_e.get_ml(lbins)  #use ml because the cfft already is a power/multiple of two maps    
    
    #auto powers
    auto_true_e = mapWF_true_cfft_e.get_cl(lbins) #use cl because we really want the power of this map
    auto_nn_e = mapNN_cfft_e.get_cl(lbins)
    auto_obs_e = mapObs_cfft_e.get_cl(lbins)
    auto_true_avg_e += auto_true_e.cl.real
    auto_nn_avg_e += auto_nn_e.cl.real
    auto_obs_avg_e += auto_obs_e.cl.real
    
    #corr coeff from spectra
    corr_coeff_nn_e = cross_power_nn_e.specs['cl']/(auto_nn_e.specs['cl']*auto_true_e.specs['cl'])**(1./2) 
    corr_coeff_nn_avg_e += corr_coeff_nn_e.real
    corr_coeff_obs_e = cross_power_obs_e.specs['cl']/(auto_obs_e.specs['cl']*auto_true_e.specs['cl'])**(1./2) 
    corr_coeff_obs_avg_e += corr_coeff_obs_e.real
        
    #difference maps
    ### V1: first qu -> eb then diff
    #diff_nn_cfft_e = mapNN_cfft_e - mapWF_true_cfft_e  
    #diff_obs_cfft_e = mapObs_cfft_e - mapWF_true_cfft_e 
    ### V2: first diff then qu -> eb 
    diff_nn_q = mapNN_map_q - mapWF_true_map_q
    diff_nn_u = mapNN_map_u - mapWF_true_map_u
    tqumap_diff_nn = ql.maps.tqumap( nx, dx, maps=[np.zeros((nx,nx)),diff_nn_q,diff_nn_u] )
    tebfft_diff_nn = tqumap_diff_nn.get_teb() 
    diff_nn_cfft_e = tebfft_diff_nn.get_cffts()[1]    
    diff_obs_q = mapObs_map_q - mapWF_true_map_q
    diff_obs_u = mapObs_map_u - mapWF_true_map_u
    tqumap_diff_obs = ql.maps.tqumap( nx, dx, maps=[np.zeros((nx,nx)),diff_obs_q,diff_obs_u] )
    tebfft_diff_obs = tqumap_diff_obs.get_teb() 
    diff_obs_cfft_e = tebfft_diff_obs.get_cffts()[1]    
    ###
    diffpower_nn_e = diff_nn_cfft_e.get_cl(lbins) #use cl because we really want the power of this map
    diffpower_obs_e = diff_obs_cfft_e.get_cl(lbins)
    diffpower_nn_avg_e += diffpower_nn_e.cl.real
    diffpower_obs_avg_e += diffpower_obs_e.cl.real        
    
    
    ######## B maps
    mapWF_true_map_b = data_test_input[map_id,:,:,13] 
    mapObs_map_b = data_test_input[map_id,:,:,9]  

    #make these rmaps and get cffts from which we can get cls and mls
    mapWF_true_cfft_b = ql.maps.rmap(nx, dx,map=mapWF_true_map_b).get_cfft()
    #mapNN_cfft_b = ql.maps.rmap(nx, dx,map=mapNN_map_b).get_cfft()
    mapNN_cfft_b = tebfft.get_cffts()[2]
    mapObs_cfft_b = ql.maps.rmap(nx, dx,map=mapObs_map_b).get_cfft()
    
    #cross powers
    cross_map_cfft_nn_b = ql.maps.cfft( nx, dx, fft=(mapNN_cfft_b.fft * np.conj(mapWF_true_cfft_b.fft)) )
    cross_power_nn_b = cross_map_cfft_nn_b.get_ml(lbins)  #use ml because the cfft already is a power/multiple of two maps    
    cross_map_cfft_obs_b = ql.maps.cfft( nx, dx, fft=(mapObs_cfft_b.fft * np.conj(mapWF_true_cfft_b.fft)) )
    cross_power_obs_b = cross_map_cfft_obs_b.get_ml(lbins)  #use ml because the cfft already is a power/multiple of two maps    
    
    #auto powers
    auto_true_b = mapWF_true_cfft_b.get_cl(lbins) #use cl because we really want the power of this map
    auto_nn_b = mapNN_cfft_b.get_cl(lbins)
    auto_obs_b = mapObs_cfft_b.get_cl(lbins)
    auto_true_avg_b += auto_true_b.cl.real
    auto_nn_avg_b += auto_nn_b.cl.real
    auto_obs_avg_b += auto_obs_b.cl.real
    
    #corr coeff from spectra
    corr_coeff_nn_b = cross_power_nn_b.specs['cl']/(auto_nn_b.specs['cl']*auto_true_b.specs['cl'])**(1./2) 
    corr_coeff_nn_avg_b += corr_coeff_nn_b.real
    corr_coeff_obs_b = cross_power_obs_b.specs['cl']/(auto_obs_b.specs['cl']*auto_true_b.specs['cl'])**(1./2) 
    corr_coeff_obs_avg_b += corr_coeff_obs_b.real
        
    #difference maps
    ### V1: first qu -> eb then diff
    #diff_nn_cfft_b = mapNN_cfft_b - mapWF_true_cfft_b  
    #diff_obs_cfft_b = mapObs_cfft_b - mapWF_true_cfft_b
    ### V2: first diff then qu -> eb 
    diff_nn_cfft_b = tebfft_diff_nn.get_cffts()[2]
    diff_obs_cfft_b = tebfft_diff_obs.get_cffts()[2]
    ### 
    diffpower_nn_b = diff_nn_cfft_b.get_cl(lbins) #use cl because we really want the power of this map
    diffpower_obs_b = diff_obs_cfft_b.get_cl(lbins)
    diffpower_nn_avg_b += diffpower_nn_b.cl.real
    diffpower_obs_avg_b += diffpower_obs_b.cl.real        
        
    
#averages
corr_coeff_nn_avg_q = corr_coeff_nn_avg_q/nsims
corr_coeff_obs_avg_q = corr_coeff_obs_avg_q/nsims
auto_nn_avg_q = auto_nn_avg_q/nsims
auto_true_avg_q = auto_true_avg_q/nsims
auto_obs_avg_q = auto_obs_avg_q/nsims
diffpower_nn_avg_q = diffpower_nn_avg_q/nsims
diffpower_obs_avg_q = diffpower_obs_avg_q/nsims

corr_coeff_nn_avg_u = corr_coeff_nn_avg_u/nsims
corr_coeff_obs_avg_u = corr_coeff_obs_avg_u/nsims
auto_nn_avg_u = auto_nn_avg_u/nsims
auto_true_avg_u = auto_true_avg_u/nsims
auto_obs_avg_u = auto_obs_avg_u/nsims
diffpower_nn_avg_u = diffpower_nn_avg_u/nsims
diffpower_obs_avg_u = diffpower_obs_avg_u/nsims

corr_coeff_nn_avg_e = corr_coeff_nn_avg_e/nsims
corr_coeff_obs_avg_e = corr_coeff_obs_avg_e/nsims
auto_nn_avg_e = auto_nn_avg_e/nsims
auto_true_avg_e = auto_true_avg_e/nsims
auto_obs_avg_e = auto_obs_avg_e/nsims
diffpower_nn_avg_e = diffpower_nn_avg_e/nsims
diffpower_obs_avg_e = diffpower_obs_avg_e/nsims

corr_coeff_nn_avg_b = corr_coeff_nn_avg_b/nsims
corr_coeff_obs_avg_b = corr_coeff_obs_avg_b/nsims
auto_nn_avg_b = auto_nn_avg_b/nsims
auto_true_avg_b = auto_true_avg_b/nsims
auto_obs_avg_b = auto_obs_avg_b/nsims
diffpower_nn_avg_b = diffpower_nn_avg_b/nsims
diffpower_obs_avg_b = diffpower_obs_avg_b/nsims


In [ ]:
#Q map cross correlation combi plot
fig=plt.figure(figsize=(12,8))
fig.suptitle('Q maps', fontsize=16)

ax1=fig.add_subplot(221)
ax1.plot(ell_binned,corr_coeff_nn_avg_q,color='red',label='$r_{WF,NN}$')
ax1.plot(ell_binned,corr_coeff_obs_avg_q,color='blue',label='$r_{WF,Obs}$')
plt.legend(loc=1,frameon=False,fontsize=14)
ax1.set_ylabel('r', fontsize=20)
ax2=fig.add_subplot(223)
ax2.plot(ell_binned,1-corr_coeff_nn_avg_q,color='red',label='$r_{WF,NN}$')
ax2.plot(ell_binned,1-corr_coeff_obs_avg_q,color='blue',label='$r_{WF,Obs}$')
ax2.set_yscale('log')
ax2.set_ylabel('1-r', fontsize=20)
ax2.set_xlabel('$\ell$', fontsize=20)
plt.legend(loc=1,frameon=False,fontsize=14)

#cl power
ax = fig.add_subplot(222)
ax.plot(ell_binned,auto_nn_avg_q*ell_binned**2.,color='red',label='$C_\ell^{NN}$')
ax.plot(ell_binned,auto_true_avg_q*ell_binned**2.,color='black',label=r'$C_\ell^{WF}$')
ax.plot(ell_binned,auto_obs_avg_q*ell_binned**2.,color='blue',label=r'$C_\ell^{obs}$')
ax.set_ylabel('$\ell^2 C_\ell$', fontsize=20)
plt.legend(loc=1,frameon=False,fontsize=14)
ax.set_yscale('log')

#diff power
ax2=fig.add_subplot(224)
ax2.plot(ell_binned,diffpower_nn_avg_q/auto_true_avg_q,color='red',label='$\Delta_\ell^{NN}$')
ax2.plot(ell_binned,diffpower_obs_avg_q/auto_true_avg_q,color='blue',label=r'$\Delta_\ell^{obs}$')
ax2.set_yscale('log')
ax2.set_xlabel('$\ell$', fontsize=20)
ax2.set_ylabel('$\Delta_\ell$', fontsize=20)
plt.legend(loc=1,frameon=False,fontsize=14)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

fig.savefig(params.folder_path_run+"quality_measures_q.pdf")

In [ ]:
#U map cross correlation combi plot
fig=plt.figure(figsize=(12,8))
fig.suptitle('U maps', fontsize=16)

ax1=fig.add_subplot(221)
ax1.plot(ell_binned,corr_coeff_nn_avg_u,color='red',label='$r_{WF,NN}$')
ax1.plot(ell_binned,corr_coeff_obs_avg_u,color='blue',label='$r_{WF,Obs}$')
plt.legend(loc=1,frameon=False,fontsize=14)
ax1.set_ylabel('r', fontsize=20)
ax2=fig.add_subplot(223)
ax2.plot(ell_binned,1-corr_coeff_nn_avg_u,color='red',label='$r_{WF,NN}$')
ax2.plot(ell_binned,1-corr_coeff_obs_avg_u,color='blue',label='$r_{WF,Obs}$')
ax2.set_yscale('log')
ax2.set_ylabel('1-r', fontsize=20)
ax2.set_xlabel('$\ell$', fontsize=20)
plt.legend(loc=1,frameon=False,fontsize=14)

#cl power
ax = fig.add_subplot(222)
ax.plot(ell_binned,auto_nn_avg_u*ell_binned**2.,color='red',label='$C_\ell^{NN}$')
ax.plot(ell_binned,auto_true_avg_u*ell_binned**2.,color='black',label=r'$C_\ell^{WF}$')
ax.plot(ell_binned,auto_obs_avg_u*ell_binned**2.,color='blue',label=r'$C_\ell^{obs}$')
ax.set_ylabel('$\ell^2 C_\ell$', fontsize=20)
plt.legend(loc=1,frameon=False,fontsize=14)
ax.set_yscale('log')

#diff power
ax2=fig.add_subplot(224)
ax2.plot(ell_binned,diffpower_nn_avg_u/auto_true_avg_u,color='red',label='$\Delta_\ell^{NN}$')
ax2.plot(ell_binned,diffpower_obs_avg_u/auto_true_avg_u,color='blue',label=r'$\Delta_\ell^{obs}$')
ax2.set_yscale('log')
ax2.set_xlabel('$\ell$', fontsize=20)
ax2.set_ylabel('$\Delta_\ell$', fontsize=20)
plt.legend(loc=1,frameon=False,fontsize=14)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

fig.savefig(params.folder_path_run+"quality_measures_u.pdf")

In [ ]:
cl_ql = ql.spec.get_camb_lensedcl(lmax=params.lmax) 
ell_ql = np.arange(0,cl_ql.cltt.shape[0])
cl_ee_ql = cl_ql.clee
cl_bb_ql = cl_ql.clbb #cl_ql.clee TEST
bl = ql.spec.bl(fwhm_arcmin=params.fwhm_arcmin, lmax=params.lmax)
nl_ee = (params.nlev_p*np.pi/180./60.)**2 #/ bl**2
nl_bb = (params.nlev_p*np.pi/180./60.)**2 #/ bl**2

#E map cross correlation combi plot
fig=plt.figure(figsize=(12,8))
fig.suptitle('E modes', fontsize=16)

ax1=fig.add_subplot(221)
ax1.plot(ell_binned,corr_coeff_nn_avg_e,color='red',label='$r_{WF,NN}$')
ax1.plot(ell_binned,corr_coeff_obs_avg_e,color='blue',label='$r_{WF,Obs}$')
plt.legend(loc=1,frameon=False,fontsize=14)
ax1.set_ylabel('r', fontsize=20)
ax2=fig.add_subplot(223)
ax2.plot(ell_binned,1-corr_coeff_nn_avg_e,color='red',label='$r_{WF,NN}$')
ax2.plot(ell_binned,1-corr_coeff_obs_avg_e,color='blue',label='$r_{WF,Obs}$')
ax2.set_yscale('log')
ax2.set_ylabel('1-r', fontsize=20)
ax2.set_xlabel('$\ell$', fontsize=20)
plt.legend(loc=1,frameon=False,fontsize=14)

#cl power
ax = fig.add_subplot(222)
ax.plot(ell_binned,auto_nn_avg_e*ell_binned**2.,color='red',label='$C_\ell^{NN}$')
ax.plot(ell_binned,auto_true_avg_e*ell_binned**2.,color='black',label=r'$C_\ell^{WF}$')
ax.plot(ell_ql,cl_ee_ql*ell_ql**2.,color='green',label=r'$C_\ell^{sky}$')
ax.plot(ell_ql,nl_ee*ell_ql**2.,color='green',ls='dashed',label=r'$N_\ell$')
ax.plot(ell_binned,auto_obs_avg_e*ell_binned**2.,color='blue',ls='dashed',label=r'$C_\ell^{obs}$')
ax.set_ylabel('$\ell^2 C_\ell$', fontsize=20)
ax.set_xlim(0,lmax)
plt.legend(loc=1,frameon=False,fontsize=14)
ax.set_yscale('log')

#diff power
ax2=fig.add_subplot(224)
ax2.plot(ell_binned,diffpower_nn_avg_e/auto_true_avg_e,color='red',label='$\Delta_\ell^{NN}$')
ax2.plot(ell_binned,diffpower_obs_avg_e/auto_true_avg_e,color='blue',label=r'$\Delta_\ell^{obs}$')
ax2.set_yscale('log')
ax2.set_xlabel('$\ell$', fontsize=20)
ax2.set_ylabel('$\Delta_\ell$', fontsize=20)
plt.legend(loc=1,frameon=False,fontsize=14)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

fig.savefig(params.folder_path_run+"quality_measures_e.pdf")
fig.savefig("quality_measures_e.pdf")

In [ ]:
#B map cross correlation combi plot
fig=plt.figure(figsize=(12,8))
fig.suptitle('B modes', fontsize=16)

ax1=fig.add_subplot(221)
ax1.plot(ell_binned,corr_coeff_nn_avg_b,color='red',label='$r_{WF,NN}$')
ax1.plot(ell_binned,corr_coeff_obs_avg_b,color='blue',label='$r_{WF,Obs}$')
plt.legend(loc=1,frameon=False,fontsize=14)
ax1.set_ylabel('r', fontsize=20)
ax2=fig.add_subplot(223)
ax2.plot(ell_binned,1-corr_coeff_nn_avg_b,color='red',label='$r_{WF,NN}$') #,marker="."
ax2.plot(ell_binned,1-corr_coeff_obs_avg_b,color='blue',label='$r_{WF,Obs}$')
ax2.set_yscale('log')
ax2.set_ylabel('1-r', fontsize=20)
ax2.set_xlabel('$\ell$', fontsize=20)
plt.legend(loc=1,frameon=False,fontsize=14)

#cl power
ax = fig.add_subplot(222)
ax.plot(ell_binned,auto_nn_avg_b*ell_binned**2.,color='red',label='$C_\ell^{NN}$')
ax.plot(ell_binned,auto_true_avg_b*ell_binned**2.,color='black',label=r'$C_\ell^{WF}$')
ax.plot(ell_binned,auto_obs_avg_b*ell_binned**2.,color='blue',ls='dashed',label=r'$C_\ell^{obs}$')
ax.plot(ell_ql,cl_bb_ql*ell_ql**2.,color='green',label=r'$C_\ell^{sky}$')
ax.plot(ell_ql,nl_bb*ell_ql**2.,color='green',ls='dashed',label=r'$N_\ell$')
ax.set_ylabel('$\ell^2 C_\ell$', fontsize=20)
ax.set_xlim(0,lmax)
plt.legend(loc=1,frameon=False,fontsize=14)
ax.set_yscale('log')

#diff power
ax2=fig.add_subplot(224)
ax2.plot(ell_binned,diffpower_nn_avg_b/auto_true_avg_b,color='red',label='$\Delta_\ell^{NN}$')
ax2.plot(ell_binned,diffpower_obs_avg_b/auto_true_avg_b,color='blue',label=r'$\Delta_\ell^{obs}$')
ax2.set_yscale('log')
ax2.set_xlabel('$\ell$', fontsize=20)
ax2.set_ylabel('$\Delta_\ell$', fontsize=20)
plt.legend(loc=1,frameon=False,fontsize=14)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

fig.savefig(params.folder_path_run+"quality_measures_b.pdf")
fig.savefig("quality_measures_b.pdf")

In [ ]:
#E B map cross correlation combi plot
fig=plt.figure(figsize=(12,8))
#fig.suptitle('E and B modes', fontsize=16)

# ax1=fig.add_subplot(221)
# ax1.plot(ell_binned,corr_coeff_nn_avg_e,color='red',label='$r_{WF,E}$')
# ax1.plot(ell_binned,corr_coeff_nn_avg_b,color='blue',label='$r_{WF,B}$')
# plt.legend(loc=1,frameon=False,fontsize=14)
# ax1.set_ylabel('r', fontsize=20)

ax2=fig.add_subplot(221)
ax2.plot(ell_binned,1-corr_coeff_nn_avg_e,color='red',label='$r_{WF,E}$')
ax2.plot(ell_binned,1-corr_coeff_nn_avg_b,color='blue',label='$r_{WF,B}$')
ax2.set_yscale('log')
ax2.set_ylabel('1-r', fontsize=20)
ax2.set_xlabel('$\ell$', fontsize=20)
ax2.set_ylim(10**(-4.),1)
plt.legend(loc=1,frameon=False,fontsize=14)

#cl power
ax = fig.add_subplot(223)
ax.plot(ell_binned,auto_nn_avg_e*ell_binned**2.,color='red',label='$C_\ell^{E,NN}$')
ax.plot(ell_binned,auto_true_avg_e*ell_binned**2.,color='black',label=r'$C_\ell^{E,WF}$')
ax.plot(ell_ql,cl_ee_ql*ell_ql**2.,color='green',label=r'$C_\ell^{E,sky}$')
ax.plot(ell_ql,nl_ee*ell_ql**2.,color='green',ls='dashed',label=r'$N_\ell$')
ax.plot(ell_binned,auto_obs_avg_e*ell_binned**2.,color='blue',ls='dashed',label=r'$C_\ell^{E,obs}$')
ax.set_ylabel('$\ell^2 C_\ell^E$', fontsize=20)
ax.set_xlim(0,lmax)
plt.legend(loc=1,frameon=False,fontsize=14)
ax.set_yscale('log')

ax = fig.add_subplot(224)
ax.plot(ell_binned,auto_nn_avg_b*ell_binned**2.,color='red',label='$C_\ell^{B,NN}$')
ax.plot(ell_binned,auto_true_avg_b*ell_binned**2.,color='black',label=r'$C_\ell^{B,WF}$')
ax.plot(ell_ql,cl_bb_ql*ell_ql**2.,color='green',label=r'$C_\ell^{B,sky}$')
ax.plot(ell_ql,nl_bb*ell_ql**2.,color='green',ls='dashed',label=r'$N_\ell$')
ax.plot(ell_binned,auto_obs_avg_b*ell_binned**2.,color='blue',ls='dashed',label=r'$C_\ell^{B,obs}$')
ax.set_ylabel('$\ell^2 C_\ell^B$', fontsize=20)
ax.set_xlim(0,lmax)
plt.legend(loc=1,frameon=False,fontsize=14)
ax.set_yscale('log')

#diff power
ax2=fig.add_subplot(222)
ax2.plot(ell_binned,diffpower_nn_avg_e/auto_true_avg_e,color='red',label='$\Delta_\ell^{E}$')
ax2.plot(ell_binned,diffpower_nn_avg_b/auto_true_avg_b,color='blue',label=r'$\Delta_\ell^{B}$')
ax2.set_yscale('log')
ax2.set_xlabel('$\ell$', fontsize=20)
ax2.set_ylabel('$\Delta_\ell$', fontsize=20)
ax2.set_ylim(10**(-4.),1)
plt.legend(loc=1,frameon=False,fontsize=14)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

fig.savefig(params.folder_path_run+"quality_measures_eb.pdf")
fig.savefig("quality_measures_eb.pdf")

In [ ]:
#save plot data
np.savez("data_eb.npz",ell_binned=ell_binned,r_e=1-corr_coeff_nn_avg_e,diff_e=diffpower_nn_avg_e/auto_true_avg_e,r_b=1-corr_coeff_nn_avg_b,diff_b=diffpower_nn_avg_b/auto_true_avg_b)

In [ ]:
# QU plots
fig, ax = plt.subplots(2, 2, figsize=(12, 12))
    
ax[1,0].imshow(data_test_result[map_id,:,:,0]) #, vmax=vmax, vmin=-vmax
ax[1,0].set_title(r'Q map WF NN', fontsize=20)    

ax[1,1].imshow(data_test_input[map_id,:,:,14])
ax[1,1].set_title(r'Q map WF exact', fontsize=20)

ax[0,1].imshow(data_test_input[map_id,:,:,10])
ax[0,1].set_title(r'Q map observed', fontsize=20)

ax[0,0].imshow(data_test_input[map_id,:,:,6])
ax[0,0].set_title(r'Q map sky', fontsize=20)

fig.tight_layout()
fig.savefig(params.folder_path_run+"maps_comparison_Q.pdf")


fig, ax = plt.subplots(2, 2, figsize=(12, 12))

ax[1,0].imshow(data_test_result[map_id,:,:,1]) #, vmax=vmax, vmin=-vmax
ax[1,0].set_title(r'U map WF NN', fontsize=20)    

ax[1,1].imshow(data_test_input[map_id,:,:,15])
ax[1,1].set_title(r'U map WF exact', fontsize=20)

ax[0,1].imshow(data_test_input[map_id,:,:,11])
ax[0,1].set_title(r'U map observed', fontsize=20)

ax[0,0].imshow(data_test_input[map_id,:,:,7])
ax[0,0].set_title(r'U map sky', fontsize=20)

fig.tight_layout()
fig.savefig(params.folder_path_run+"maps_comparison_U.pdf")

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 12))

ax[1,0].imshow(data_test_result_eb[map_id,:,:,0]) #, vmax=vmax, vmin=-vmax
ax[1,0].set_title(r'E map WF NN', fontsize=20)    

ax[1,1].imshow(data_test_input[map_id,:,:,12])
ax[1,1].set_title(r'E map WF exact', fontsize=20)

ax[0,1].imshow(data_test_input[map_id,:,:,8])
ax[0,1].set_title(r'E map observed', fontsize=20)

ax[0,0].imshow(data_test_input[map_id,:,:,4])
ax[0,0].set_title(r'E map sky', fontsize=20)

fig.tight_layout()
fig.savefig(params.folder_path_run+"maps_comparison_E.pdf")


fig, ax = plt.subplots(2, 2, figsize=(12, 12))

ax[1,0].imshow(data_test_result_eb[map_id,:,:,1]) #, vmax=vmax, vmin=-vmax
ax[1,0].set_title(r'B map WF NN', fontsize=20)    

ax[1,1].imshow(data_test_input[map_id,:,:,13])
ax[1,1].set_title(r'B map WF exact', fontsize=20)

ax[0,1].imshow(data_test_input[map_id,:,:,9])
ax[0,1].set_title(r'B map observed', fontsize=20)

ax[0,0].imshow(data_test_input[map_id,:,:,5])
ax[0,0].set_title(r'B map sky', fontsize=20)

fig.tight_layout()
fig.savefig(params.folder_path_run+"maps_comparison_B.pdf")

In [ ]:
# QU plots
fig, ax = plt.subplots(2, 2, figsize=(12, 12))
    
ax[0,0].imshow(data_test_result[map_id,:,:,0]) #, vmax=vmax, vmin=-vmax
ax[0,0].set_title(r'Q map WF NN', fontsize=20)    

ax[0,1].imshow(data_test_input[map_id,:,:,14])
ax[0,1].set_title(r'Q map WF exact', fontsize=20)

ax[1,0].imshow(data_test_result[map_id,:,:,1]) #, vmax=vmax, vmin=-vmax
ax[1,0].set_title(r'U map WF NN', fontsize=20)    

ax[1,1].imshow(data_test_input[map_id,:,:,15])
ax[1,1].set_title(r'U map WF exact', fontsize=20)

fig.tight_layout()
fig.savefig(params.folder_path_run+"maps_comparison_QU.pdf")
fig.savefig("maps_comparison_QU.pdf")

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 12))

ax[0,0].imshow(data_test_result_eb[map_id,:,:,0]) #, vmax=vmax, vmin=-vmax
ax[0,0].set_title(r'E map WF NN', fontsize=20)    

ax[0,1].imshow(data_test_input[map_id,:,:,12])
ax[0,1].set_title(r'E map WF exact', fontsize=20)

ax[1,0].imshow(data_test_result_eb[map_id,:,:,1]) #, vmax=vmax, vmin=-vmax
ax[1,0].set_title(r'B map WF NN', fontsize=20)    

ax[1,1].imshow(data_test_input[map_id,:,:,13])
ax[1,1].set_title(r'B map WF exact', fontsize=20)

fig.tight_layout()
fig.savefig(params.folder_path_run+"maps_comparison_EB.pdf")

# plot the loss

In [ ]:
folder_path_run = params.datapath+"runs/n128_qu_j3_adam04_500epochs_backupafter500/"
data = np.load(folder_path_run+"loss.npz")
loss_j3 = data['loss']
val_loss_j3 = data['val_loss']

folder_path_run = params.datapath+"runs/n128_qu_j2_adam04_500epochs/"
data = np.load(folder_path_run+"loss.npz")
loss_j2 = data['loss']
val_loss_j2 = data['val_loss']

folder_path_run = params.datapath+"runs/n128_qu_j4_adam04_500epochs/"
data = np.load(folder_path_run+"loss.npz")
loss_j4 = data['loss']
val_loss_j4 = data['val_loss']

In [ ]:
#for combi plot:
#corr_coeff_nn_avg_e_j2 = np.copy(corr_coeff_nn_avg_e)
#corr_coeff_nn_avg_e_j3 = np.copy(corr_coeff_nn_avg_e)
#corr_coeff_nn_avg_e_j4 = np.copy(corr_coeff_nn_avg_e)

In [ ]:
epochs_range = range(len(loss_j3))
epoch_minid1=20
epoch_minid2=10

fig, ax = plt.subplots(1, 2, figsize=(12, 4.5))
ax[0].plot(epochs_range[epoch_minid1:], loss_j2[epoch_minid1:]/val_loss_j2[-1], label='J2 Training Loss')
ax[0].plot(epochs_range[epoch_minid1:], val_loss_j2[epoch_minid1:]/val_loss_j2[-1], label='J2 Validation Loss')
ax[0].plot(epochs_range[epoch_minid2:], loss_j3[epoch_minid2:]/val_loss_j3[-1], label='J3 Training Loss')
ax[0].plot(epochs_range[epoch_minid2:], val_loss_j3[epoch_minid2:]/val_loss_j3[-1], label='J3 Validation Loss')
ax[0].legend(loc='upper right',frameon=False,fontsize=12)

ax[1].plot(ell_binned,1-corr_coeff_nn_avg_e_j3,color='red',label='$r_{J3}$')
ax[1].plot(ell_binned,1-corr_coeff_nn_avg_e_j2,color='blue',label='$r_{J2}$')
ax[1].plot(ell_binned,1-corr_coeff_nn_avg_e_j4,color='green',label='$r_{J4}$',ls="dashed")
ax[1].set_yscale('log')
ax[1].set_ylabel('1-r', fontsize=20)
ax[1].set_xlabel('$\ell$', fontsize=20)
ax[1].legend(loc='upper left',frameon=False,fontsize=12)


fig.tight_layout()
plt.show()
fig.savefig("figs/loss_compared1.pdf")

In [ ]:
epochs_range = range(len(loss))
epoch_minid=20

fig, ax = plt.subplots(1, 3, figsize=(12, 3.5))
ax[0].plot(epochs_range[epoch_minid:], loss_j3[epoch_minid:]/val_loss_j3[-1], label='J3 Training Loss')
ax[0].plot(epochs_range[epoch_minid:], val_loss_j3[epoch_minid:]/val_loss_j3[-1], label='J3 Validation Loss')
ax[0].legend(loc='upper right',frameon=False,fontsize=12)
ax[1].plot(epochs_range[epoch_minid:], loss_j2[epoch_minid:]/val_loss_j2[-1], label='J2 Training Loss')
ax[1].plot(epochs_range[epoch_minid:], val_loss_j2[epoch_minid:]/val_loss_j2[-1], label='J2 Validation Loss')
ax[1].legend(loc='upper right',frameon=False,fontsize=12)
ax[2].plot(ell_binned,1-corr_coeff_nn_avg_e,color='red',label='$r_{WF,NN}$')
ax[2].plot(ell_binned,1-corr_coeff_obs_avg_e,color='blue',label='$r_{WF,Obs}$')
ax[2].set_yscale('log')
ax[2].set_ylabel('1-r', fontsize=20)
ax[2].set_xlabel('$\ell$', fontsize=20)
ax[2].legend(loc='upper right',frameon=False,fontsize=12)
fig.tight_layout()
#plt.title('Training and Validation Loss')
plt.show()
fig.savefig("figs/loss_compared2.pdf")